In [1]:
"""
Convert AirSim's airsim_rec.txt to a clean CSV for
ego-UAV short-term trajectory prediction.

Directory layout expected:
[root]/
├─ raw_data/
│   ├─ airsim_rec.txt
│   └─ images/
│       └─ img_SimpleFlight__0_<timestamp>.png
│
└─ data_collector.py   <-- this file

Note: After running airsim API (which have import airsim), there is a high change ipynb/juypter notebook will not work, 
because there is a conflict between `airsim` and `ipykernel` module. So you need to reinstall it by: `pip install ipykernel`.
"""

from pathlib import Path
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

# ----------------------- paths -----------------------
ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
RAW_DIR    = ROOT / "raw_data"
TXT_FILE   = RAW_DIR / "airsim_rec.txt"
OUT_CSV    = RAW_DIR / "airsim_trajectory.csv"


In [2]:
# -------------------- load .txt ----------------------
df = pd.read_csv(
    TXT_FILE,
    sep=r"\t+",           # tab‑delimited
    engine="python",
    comment="#"
)

# ----------------- drop vehicle name ----------------
df = df.drop(columns=["VehicleName"], errors="ignore")

# ------------- quaternion ➜ roll‑pitch‑yaw ----------
def quat_to_rpy(qw, qx, qy, qz):
    # scipy expects (x, y, z, w)
    r = R.from_quat([qx, qy, qz, qw])
    return r.as_euler("xyz", degrees=False)   # roll, pitch, yaw

rpy = np.vstack([
    quat_to_rpy(qw, qx, qy, qz)
    for qw, qx, qy, qz in df[["Q_W","Q_X","Q_Y","Q_Z"]].values
])
df[["ROLL","PITCH","YAW"]] = rpy

# optional: drop quaternion columns
df = df.drop(columns=["Q_W","Q_X","Q_Y","Q_Z"])

# ---------- add sin/cos of yaw for continuity -------
df["YAW_SIN"] = np.sin(df["YAW"])
df["YAW_COS"] = np.cos(df["YAW"])

# ----------------- compute velocity -----------------
# AirSim TimeStamp is in micro‑seconds
df["TimeSec"] = df["TimeStamp"] * 1e-6

for axis in ["X","Y","Z"]:
    df[f"VEL_{axis}"] = df[f"POS_{axis}"].diff() / df["TimeSec"].diff()

# first row has NaNs after diff – drop or fill
df = df.dropna().reset_index(drop=True)

# --------------- save cleaned CSV -------------------
df.to_csv(OUT_CSV, index=False)
print(f"[✓]  Saved {len(df):,} rows to {OUT_CSV.relative_to(ROOT)}")


[✓]  Saved 217 rows to raw_data\airsim_trajectory.csv


In [3]:
# ------------------------------------------------------------------
# 6. Quick sanity check (optional)
# ------------------------------------------------------------------
print(df.head(5))


       TimeStamp    POS_X     POS_Y    POS_Z  \
0  1750186579775  1.14191  0.916024 -30.2011   
1  1750186580276  2.18092  1.725050 -30.0982   
2  1750186580771  3.33536  2.531490 -29.9403   
3  1750186581284  4.38019  3.174380 -30.0801   
4  1750186581770  4.41093  2.997450 -30.7159   

                                     ImageFile      ROLL     PITCH       YAW  \
0  img_SimpleFlight__0_1750186579850130800.png  0.095935 -0.118069  0.000102   
1  img_SimpleFlight__0_1750186580347858500.png  0.003593 -0.062322  0.008840   
2  img_SimpleFlight__0_1750186580847827300.png  0.002255 -0.037457  0.009552   
3  img_SimpleFlight__0_1750186581359899800.png -0.306492  0.368516 -0.045244   
4  img_SimpleFlight__0_1750186581845470200.png -0.189929  0.232335 -0.005169   

    YAW_SIN   YAW_COS       TimeSec        VEL_X        VEL_Y        VEL_Z  
0  0.000102  1.000000  1.750187e+06  1706.641646  1366.502371  -423.383049  
1  0.008840  0.999961  1.750187e+06  2073.872849  1614.822817   205.389280  